In [ ]:
import pathlib

import geopandas as gpd
import pandas as pd
import panel as pn
import pydeck as pdk

pn.extension("deckgl")

In [ ]:
# See data/README.md to download this file
DATA_ROOT_PATH = pathlib.Path().absolute().parent / "data"

In [ ]:
community_next_globus_info = gpd.read_parquet(
    DATA_ROOT_PATH / "processed" / "german_communities.parquet",
)
community_next_globus_info["distance_km"] = round(community_next_globus_info["distance"] / 1000, 2)
community_next_globus_info

In [ ]:
globus_positions = pd.read_csv(DATA_ROOT_PATH / "processed" / "globus_info.csv", sep=";")
globus_positions["geometry"] = list(
    zip(globus_positions["lon"], globus_positions["lat"], strict=False),
)
globus_positions

In [ ]:
INITIAL_VIEW_STATE = pdk.ViewState(latitude=51.3, longitude=10, zoom=6, max_zoom=16, bearing=0)


geojson = pdk.Layer(
    "GeoJsonLayer",
    community_next_globus_info,
    opacity=0.5,
    stroked=True,
    filled=True,
    get_fill_color=[
        "(distance_km / 200) * 255",
        "255 - ((distance_km / 200) * 255)",
        "(distance_km / 200) * 255",
    ],
    get_line_color=[0, 0, 0],
    auto_highlight=True,
    pickable=True,
)
point_layer = pdk.Layer(
    "ScatterplotLayer",
    globus_positions,
    opacity=0.8,
    stroked=True,
    filled=True,
    get_fill_color=[255, 255, 255],
    get_position="geometry",
    radius_scale=1,
    radius_min_pixels=3,
    radius_max_pixels=10,
    get_radius=10,
    line_width_min_pixels=1,
)

tooltip = {"text": "Community: {community}\nNext: {name}, {location}\nDistance: {distance_km} km"}

pdk_deck = pdk.Deck(
    layers=[geojson, point_layer],
    initial_view_state=INITIAL_VIEW_STATE,
    tooltip=tooltip,
    map_style="",
    map_provider=None,
)

pn.pane.DeckGL(pdk_deck, sizing_mode="stretch_both", min_height=1200).servable()